In [1]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale 
from sklearn import model_selection
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor

from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
hit = pd.read_csv("Hitters.csv")
df = hit.copy()
df = df.dropna()
dms = pd.get_dummies(df[['League', 'Division', 'NewLeague']])
y = df["Salary"]
X_ = df.drop(['Salary', 'League', 'Division', 'NewLeague'], axis=1).astype('float64')
X = pd.concat([X_, dms[['League_N', 'Division_W', 'NewLeague_N']]], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.25, 
                                                    random_state=42)

In [3]:
from sklearn.preprocessing import StandardScaler

In [4]:
scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler()

In [13]:
X_train_scaled = scaler.transform(X_train)
X_test_scaled= scaler.transform(X_test)

In [14]:
from sklearn.neural_network import MLPRegressor

In [15]:
mlp_model = MLPRegressor(hidden_layer_sizes = (100,20)).fit(X_train_scaled, y_train) 
## bu sekilde gizli katman sayısını ayarlarsın ama direkte yapabilirisn
#mlp_model = MLPRegressor().fit(X_train_scaled, y_train) 

In [43]:
mlp_model.get_params()

{'activation': 'relu',
 'alpha': 0.0001,
 'batch_size': 'auto',
 'beta_1': 0.9,
 'beta_2': 0.999,
 'early_stopping': False,
 'epsilon': 1e-08,
 'hidden_layer_sizes': (100, 20),
 'learning_rate': 'constant',
 'learning_rate_init': 0.001,
 'max_fun': 15000,
 'max_iter': 200,
 'momentum': 0.9,
 'n_iter_no_change': 10,
 'nesterovs_momentum': True,
 'power_t': 0.5,
 'random_state': None,
 'shuffle': True,
 'solver': 'adam',
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': False,
 'warm_start': False}

In [17]:
mlp_model.n_layers_

4

In [18]:
mlp_model.hidden_layer_sizes

(100, 20)

In [19]:
### TAHMİN

In [20]:
y_pred = mlp_model.predict(X_test_scaled)

In [21]:
np.sqrt(mean_squared_error(y_test, y_pred))

474.9567042099803

In [22]:
####model tuning

In [35]:
mlp_params = {'alpha': [0.1, 0.01,0.02,0.005],
             'hidden_layer_sizes': [(20,20),(100,50,150),(300,200,150)],
             'activation': ['relu','logistic']} ## logistic koymasanda olur denemek için koydum
"""
1. alpha: [0.1, 0.01, 0.02, 0.005]
alpha, MLP modelinde L2 regularization (ridge) için kullanılan terimdir.
Bu, modelin aşırı uyum yapmasını (overfitting) engellemeye yardımcı olur.
Bu parametre, küçük değerler için az regularization, büyük değerler için daha güçlü regularization anlamına gelir.
Bu liste, denenmesi gereken alpha değerlerini gösteriyor. Her biri, regularization gücünü farklı bir seviyede uygulayacaktır.
2. hidden_layer_sizes: [(20,20), (100,50,150), (300,200,150)]
Bu parametre, gizli katmanların boyutlarını tanımlar. Her tuple, katman sayısını ve her katmandaki nöron sayısını belirtir.
(20, 20): Bu, iki gizli katmanlı bir MLP modelini temsil eder ve her katmanda 20 nöron vardır.
(100, 50, 150): Üç gizli katmanlı bir model, birinci katmanda 100, ikinci katmanda 50 ve üçüncü katmanda 150 nöron bulunur.
(300, 200, 150): Üç katmanlı başka bir model, daha büyük katmanlarla (300, 200, 150 nöron) tanımlanmıştır. 
Bu daha karmaşık bir model yapısına işaret eder.
3. activation: ['relu', 'logistic']
activation, nöronlar arasında hangi aktivasyon fonksiyonunun kullanılacağını belirtir.
Aktivasyon fonksiyonları, modelin öğrenme kapasitesini artırmak için kullanılır.
'relu': ReLU (Rectified Linear Unit) aktivasyon fonksiyonu.
Genellikle daha iyi performans gösterir ve modern sinir ağlarında yaygın olarak kullanılır.
'logistic': Sigmoid (logistic) fonksiyonu, daha klasik bir aktivasyon fonksiyonudur ve çıktı değerlerini 0 ile 1 arasında sınırlayarak çalışır.
"""

In [36]:
mlp_cv_model = GridSearchCV(mlp_model, mlp_params, cv = 10)

In [37]:
mlp_cv_model.fit(X_train_scaled, y_train)

GridSearchCV(cv=10, estimator=MLPRegressor(hidden_layer_sizes=(100, 20)),
             param_grid={'activation': ['relu', 'logistic'],
                         'alpha': [0.1, 0.01, 0.02, 0.005],
                         'hidden_layer_sizes': [(20, 20), (100, 50, 150),
                                                (300, 200, 150)]})

In [38]:
mlp_cv_model.best_params_

{'activation': 'relu', 'alpha': 0.1, 'hidden_layer_sizes': (100, 50, 150)}

In [39]:
mlp_tuned = MLPRegressor(alpha = 0.02, hidden_layer_sizes = (100,50,150))

In [40]:
mlp_tuned.fit(X_train_scaled, y_train)

MLPRegressor(alpha=0.02, hidden_layer_sizes=(100, 50, 150))

In [41]:
y_pred = mlp_tuned.predict(X_test_scaled)

In [42]:
np.sqrt(mean_squared_error(y_test, y_pred))

359.0798388116322